In [1]:
import sys; sys.path.append("../../")
from DataPreprocessing.DataPreprocessing import *
from DataExploration.DataExploration import *
from utils import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt

# Question:
    Will sales increase over upcoming years for certain game?



## 1- Expectations: 
   ?????????????????????????????????

## Read Data 

In [2]:
df = read_data(kind='dataset')
df = df.dropna()

In [3]:
# print unique values of rating
print(df['Rating'].unique())


['E' 'M' 'T' 'E10+' 'AO' 'K-A' 'RP']


In [4]:
#drop rating values with number of rows less than 100
df = df.groupby('Rating').filter(lambda x: len(x) > 100)

In [5]:
print(df['Rating'].unique())


['E' 'M' 'T' 'E10+']


In [6]:
# df=remove_common_outliers(df ,[ 'Critic_Score','Global_Sales'])

In [7]:
# read the data
df_train=read_data(kind='train')
df_val = read_data(kind='val')
selected_cols=[ 'Platform', 'Genre', 'Publisher', 'Global_Sales', 'Critic_Score',
   'Developer', 'Rating', 'Critic_Count']
df_train = df_train[selected_cols]
df_val = df_val[selected_cols]
df_train.head()

,Platform,Genre,Publisher,Global_Sales,Critic_Score,Developer,Rating,Critic_Count
0,X360,Shooter,Cave,0.06,NaN,NaN,NaN,NaN
1,WiiU,Action,Namco Bandai Games,0.06,66.0,Ganbarion,T,11.0
2,DS,Role-Playing,Konami Digital Entertainment,0.23,82.0,Kojima Productions,E10+,39.0
3,PSP,Adventure,Yeti,0.01,NaN,NaN,NaN,NaN
4,X360,Action,Ubisoft,1.59,85.0,Ubisoft Montreal,M,75.0


## Read as train and val data frames

## Handle Missing values 
    - as all features used here are categorical, mode imputation is used 
      to handle missing values.


In [8]:
# handle missing values using mode imputation
df_train =handle_missing_values(df_train, handling_method='mode_interpolate',neighbors=5)
df_val =handle_missing_values(df_val,handling_method='mode_interpolate',neighbors=5)

df_train.head()

Total Number of rows : 10031
Number of rows with missing values: 5226
Number of rows after handling missing values: 10031 and Number of missing values: 0
Total Number of rows : 3344
Number of rows with missing values: 1732
Number of rows after handling missing values: 3344 and Number of missing values: 0


,Platform,Genre,Publisher,Global_Sales,Critic_Score,Developer,Rating,Critic_Count
0,X360,Shooter,Cave,0.06,66.0,Ubisoft,E,11.0
1,WiiU,Action,Namco Bandai Games,0.06,66.0,Ganbarion,T,11.0
2,DS,Role-Playing,Konami Digital Entertainment,0.23,82.0,Kojima Productions,E10+,39.0
3,PSP,Adventure,Yeti,0.01,83.5,Ubisoft,E,57.0
4,X360,Action,Ubisoft,1.59,85.0,Ubisoft Montreal,M,75.0


## prepare features and labels

In [9]:
# global sales our target column to be predicted
# encoding(df_train)
# encoding(df_val)

y_train = df_train['Rating']
X_train = df_train.drop('Rating', axis=1)
y_val = df_val['Rating']
X_val = df_val.drop('Rating', axis=1)
X_train.head()

,Platform,Genre,Publisher,Global_Sales,Critic_Score,Developer,Critic_Count
0,X360,Shooter,Cave,0.06,66.0,Ubisoft,11.0
1,WiiU,Action,Namco Bandai Games,0.06,66.0,Ganbarion,11.0
2,DS,Role-Playing,Konami Digital Entertainment,0.23,82.0,Kojima Productions,39.0
3,PSP,Adventure,Yeti,0.01,83.5,Ubisoft,57.0
4,X360,Action,Ubisoft,1.59,85.0,Ubisoft Montreal,75.0


In [10]:
# encoding the categorical columns
X_train = encoding(X_train)
X_val = encoding(X_val)

In [11]:
X_train.isnull().sum()  


Platform        0
Genre           0
Publisher       0
Global_Sales    0
Critic_Score    0
Developer       0
Critic_Count    0
dtype: int64

# Models

In [16]:
# svm model to predict the rating
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# create the model
model = SVC(kernel='linear', C=10)

# fit the model
model.fit(X_train, y_train)
# predict the model
y_pred = model.predict(X_val)
# get the accuracy score
print(f'Accuracy Score: {accuracy_score(y_val, y_pred)}')

In [13]:
# random forest model to predict the rating
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# create the model
model = RandomForestClassifier()
# fit the model
model.fit(X_train, y_train)
# predict the model
y_pred = model.predict(X_val)


# get the accuracy score
print(f'Accuracy Score: {accuracy_score(y_val, y_pred)}')


Accuracy Score: 0.72188995215311


## 1- catboost Model

In [14]:
# !pip install catboost

In [15]:
import catboost as cat
cat_feat =[ 'Publisher', 'Developer', 'Platform','Genre','Rating']
model = cat.CatBoostRegressor(random_state=100,cat_features=cat_feat,verbose=0)
model.fit(X_train ,y_train)

ValueError: 'Rating' is not in list

### Model Accuracy

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
rmse = sqrt(mean_squared_error(y_val, y_pred))
print(rmse)

9.743736845414473


In [ ]:
print("R2 score =", round(sm.r2_score( y_val, y_pred), 2))


R2 score = 0.23


## Encoding for cat features for the coming models

### 2- ElasticNet Model

In [ ]:
regr = ElasticNet(alpha=0.1)
# Prepare input data (frequncy encoding)
regr.fit(encoding(X_train), y_train)

ElasticNet(alpha=0.1)

In [ ]:
y_predicted =regr.predict(encoding(X_val))
print("R2 score =", round(sm.r2_score( y_val, y_predicted), 2))

R2 score = 0.04


### 3- Linear Regression Model

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
y_predicted =reg.predict(X_val)
# compute accuracy
print("R2 score =", round(sm.r2_score( y_val, y_predicted), 2))



R2 score = 0.04


## 3- Comparing Expectations to collecting data and models:
       - year of release didn't affect the score at all with increase or decrease 0.12 it was expected 
         as year of release has no correlation with any of the features as we saw
         in data exploration part
       - rating (class of the game) make the score very low 0.07
       - adding critic score to the feature increase score to 0.2
       - removing outliers in both critic score and sales 0.16 (lose about 800 row)
       - removing common outliers between  critic score only 0.17
       - removing outliers in both critic score only 0.19
       - removing outliers in both critic score only 0.27 this with highest score as for very 
       few times (outliers) games with very high score has no sales and this mislead the model 
       but it's okay for very high critic score got low sales as critic not user score and we     can't know user score until the game is sold (print and see what the range of outliers in both to verify this)TODO
       - removing developer from features 0.21
       - removing publisher from features 0.22
       - so clearly combination of the gives the model more info to predict the sales
         as keeping them both retain the highest score which makes sense, publisher only 
        guarantee that the game will have high sales, similarly for developer, people mainly
        buy the games knowing who the publisher they can trust. 
        ##TODO importance of features plot to verify our conclusions 
    